This project uses GCP's VM instance to scrape reddit's RSS feed into GCP's NLP. Then the data that is scraped along with the processed text is transferred into a PostgreSQL database held on GCP's platform. Then Python is used to connect to the database, query the database and visualize the data. 

Step 1 is to create a VM instance through the compute engine tab on the GCP site
Step 2 is to allow the VM access to the NLP and begin creating a script within the VM using vim

Once the data has been scraped and loaded from within the VM, anyone with a connection to the database can query the data. 

In [ ]:
#connect to db through jupyter notebook
import getpass
mypasswd = getpass.getpass()


import psycopg2
connection = psycopg2.connect(database = 'db_name', 
                              user = 'user', 
                              host = '99.999.999.99', # Replace with SQL IP
                              password = 'password')
with connection, connection.cursor() as cursor:
    cursor.execute("SELECT * FROM ratings")
    results = cursor.fetchall()
    for row in results:
        print(row)

In [ ]:
#query db using pandas
import pandas as pd

SQL = "SELECT * "
SQL += " FROM reddit_table "
SQL += " "

print(SQL)
reddit_df = pd.read_sql(SQL,connection)
reddit_df.head()

In [ ]:
#convert date to datetime object
import datetime as dt

reddit_df['date'] = pd.to_datetime(reddit_df['date'], format ='%Y/%m/%d')

In [ ]:
#assign a net positive/negative/neutral tag to each post
reddit_df['sentiment'] = 'NEU'
reddit_df.loc[reddit_df['sent_score'] >= 0.25, 'sentiment'] = 'POS'
reddit_df.loc[reddit_df['sent_score'] <= -0.25, 'sentiment'] = 'NEG'

In [ ]:
#visualize distribution of post's sentiment
import seaborn as sns

sns.countplot(x=reddit_df["sentiment"])